In [4]:
import findspark
findspark.init('/Users/kiranrudresha/Documents/spark/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession


In [5]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [6]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [7]:
data = spark.createDataFrame([
           ('0.0', ' Hi i heard about spark spark'),
           ('1.0', 'i wish java could use case classes'),
           ('1.0', 'Logistic regression models are nead')],
    ['label', 'sentence'])

In [8]:
data.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0| Hi i heard about...|
|  1.0|i wish java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [9]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words'  )

In [10]:
token_data = tokenizer.transform(data)

In [11]:
token_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0| Hi i heard about...|[, hi, i, heard, ...|
|  1.0|i wish java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [12]:
hashingtf = HashingTF(inputCol='words', outputCol='rawfeatures')

In [13]:
featurized_data = hashingtf.transform(token_data)

In [14]:
featurized_data.show(truncate = False)

+-----+-----------------------------------+------------------------------------------+--------------------------------------------------------------------------------------+
|label|sentence                           |words                                     |rawfeatures                                                                           |
+-----+-----------------------------------+------------------------------------------+--------------------------------------------------------------------------------------+
|0.0  | Hi i heard about spark spark      |[, hi, i, heard, about, spark, spark]     |(262144,[24417,49304,73197,91137,234657,249180],[1.0,1.0,1.0,1.0,2.0,1.0])            |
|1.0  |i wish java could use case classes |[i, wish, java, could, use, case, classes]|(262144,[20719,24417,55551,116873,147765,162369,192310],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|1.0  |Logistic regression models are nead|[logistic, regression, models, are, nead] |(262144,[13671,132713,150201,167122,190884],

In [15]:
idf = IDF(inputCol='rawfeatures', outputCol='features')

In [16]:
idf_model = idf.fit(featurized_data)

In [17]:
scaled_data = idf_model.transform(featurized_data)

In [19]:
scaled_data.select('label','features').show(truncate = False)
# ready for supervised machine learning 

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[24417,49304,73197,91137,234657,249180],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,1.3862943611198906,0.6931471805599453])                           |
|1.0  |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [20]:
from pyspark.ml.feature import CountVectorizer

In [23]:
count_vec = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [22]:
df = spark.createDataFrame([
    (0,"a b c".split(" ")),
    (1,"a b b c a".split(" "))],
    ["id","words"]
)

In [25]:
count_vec.fit(df).transform(df).show(truncate = False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

